## In this quick notebook we will see a basic GNN

### Importing Libraries:

In this example we will use 2 main libraries:
*   PyTorch;
*   Torch-Geometric

In [1]:
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures